In [1]:
import pandas as pd
import json 
import os
from bs4 import BeautifulSoup
import requests
import re
import time

def get_config(home_dir):
    path = "{}/config.json".format(home_dir)

    with open(path) as f:
        config = f.readlines()[0]
    
    config_dict = json.loads(config)

    return config_dict

def get_html(url):
    wait_time = 1
    html = requests.get(url, verify=False).content.decode("utf-8") 
    soup = BeautifulSoup(re.sub("<!--|-->","", html), "lxml")  
    time.sleep(wait_time)
    return soup

config_dir = os.getcwd().replace('/notebooks', '')
config = get_config(config_dir)            

/Users/greg.bolla/.local/share/virtualenvs/nfl-stat-scraper-Jt7XuzPt/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
class Player():
    def __init__(self, player_link, year = None, week = None):
        
        self.player_link = player_link   
        self.base_url = config['base_url']
        self.full_player_url = self.base_url + player_link
        self.standarized_position_dict = {
            "QB" : {"eligible_positions" : ["QB"], 
                   "class": Quarterback}
        }
        
        player_page = self.get_player_page()
        
    
    def get_player_page(self):
        
        full_player_url = self.full_player_url
        player_html = requests.get(full_player_url, verify=False).content.decode("utf-8") 
        player_soup = BeautifulSoup(re.sub("<!--|-->","", player_html), "lxml")  
        
        return player_soup
    
    def get_position(self, position):
        
        position_dict = self.standardized_position_dict
        standardized_pos = [k for k, v in position_dict.iteritems() if position in v["eligible_positions"]][0]
        return standarized_position
    
    def get_team(self, player_page, year, week = None):
        team = None
        return team
    
    def get_yrs_experience(self, player_page, positional_class, year, team):
        yrs_with_team = 0
        yrs_career = 0 
        final
        player_stats = player_soup.find(id=final_tbl_id).tbody
        player_stat_rows = player_stats.findAll('tr')

        for row in player_stat_rows:
            try:  
                row_yr = row.find('a', href=re.compile(r'years'))['href'].split('/')[2]            
            except TypeError:   
                try:
                    row_yr = row.find('th', attrs={'data-stat' : 'year_id'}).string.split('*')[0]
                except AttributeError:
                    continue

            row_yr = int(row_yr)

            if row_yr == year:
                try:
                    current_age = int(row.find('td', attrs={'data-stat' : 'age'}).string)
                except TypeError:
                    print('age not listed in year row')

            #elif row_yr == 2016 and year == 2017:
            #    current_age = int(row.find('td', attrs={'data-stat' : 'age'}).string) + 1

            yrs_with_team, yrs_career = yrs_expo_helper(row_yr, row, team, yrs_with_team, yrs_career, year)

        try:
            current_age
        except UnboundLocalError:
            print("current_age not defined, defaulting to 22")
            current_age = 22

        #print("current age: {}".format(current_age))    
        return yrs_with_team, yrs_career, current_age

    def yrs_expo_helper(self, row_yr, row, team, yrs_with_team, yrs_career, year):
        if int(row_yr) < year:    
            yrs_career += 1

            try:        
                if str(row.find("td", attrs={'data-stat':'team'}).string) == team:
                    yrs_with_team += 1
            except AttributeError:
                print("no team found in row for row_yr {}".format(str(row_yr)))
                pass

        return yrs_with_team, yrs_career

    def get_all_pro_and_pro_bowls(self, yr_row, pro_bowls, all_pros):

        for child in yr_row.find('th', {'data-stat' : 'year_id'}).children:
            if child.find('*') is not None and child.find('*') >= 0:
                pro_bowls += 1
            if child.find('+') is not None and child.find('+') >= 0:
                all_pros += 1  

        return pro_bowls, all_pros

class Quarterback(Player):
    def __init__():
        self.main_tbl_id = "passing"
        self.other_tbl_ids = ["rushing_and_receiving", "receiving_and_rushing"]

In [3]:
home_dir = os.getcwd().replace('notebooks', '')
print(home_dir)
config = get_config(home_dir)

base_url = config['base_url']

foles = Player('/players/F/FoleNi00.htm')

player_soup = foles.get_player_page()


/Users/greg.bolla/Desktop/git-projects/nfl-stat-scraper/


/Users/greg.bolla/.local/share/virtualenvs/nfl-stat-scraper-Jt7XuzPt/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/greg.bolla/.local/share/virtualenvs/nfl-stat-scraper-Jt7XuzPt/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [4]:
class Season():
    def __init__(self, year):
        self.season = year
        config_dir = os.getcwd().replace('/notebooks', '')
        config = get_config(config_dir)        
        self.base_url = config['base_url']
        
        self.schedule_url = "{}/years/{}/games.htm".format(self.base_url, self.season)
        self.schedule_games_id = "games"
        self.schedule_page = self.get_season_schedule()
    
    def get_season_schedule(self):
        
        url = self.schedule_url
        schedule_html = requests.get(url, verify=False).content.decode("utf-8") 
        schedule_soup = BeautifulSoup(re.sub("<!--|-->","", schedule_html), "lxml")  
        
        return schedule_soup
    def get_schedule_table(self):
        page = self.schedule_page
        tbl_id = self.schedule_games_id
        schedule_table = page.find(id=tbl_id)
        
        return schedule_table
    
    def get_weekly_dict(self, games_table):
        
        all_trs = games_table.findAll('tr')
        
        game_rows = list(filter(lambda x: x.has_attr('class') == False, all_trs))
        weekly_dict = {}
        
        for game in game_rows:
            week_num = game.find("th", {"data-stat" : "week_num"}).string 

            if week_num in ['Week', None]:
                continue
            if week_num not in weekly_dict.keys():
                weekly_dict[week_num] = [game]
            else:
                weekly_dict[week_num].append(game)
        return weekly_dict
season = Season(2016)

/Users/greg.bolla/.local/share/virtualenvs/nfl-stat-scraper-Jt7XuzPt/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [5]:
games_table = season.get_schedule_table()
weekly_dict = season.get_weekly_dict(games_table)

    #print(game)
        #weekly_dict[week_num] = weekly_dict[week_num].append(game)
#weekly_dict = season.get_weekly_dict(games_table)
#print(games_rows[1])

# for game in games_rows[0:5]:
#     print("---")
#     print(game)

In [6]:
# for week, games in weekly_dict.items():
    
#     print("{}: {} games".format(week, len(games)))

games = weekly_dict["5"]
print(games[0])

<tr><th class="right " csk="5" data-stat="week_num" scope="row">5</th><td class="left " data-stat="game_day_of_week">Thu</td><td class="left " csk="2016-10-06" data-stat="game_date">October 6</td><td class="right " data-stat="gametime">8:27PM</td><td class="left " data-stat="winner"><strong><a href="/teams/crd/2016.htm">Arizona Cardinals</a></strong></td><td class="right " data-stat="game_location">@</td><td class="left " data-stat="loser"><a href="/teams/sfo/2016.htm">San Francisco 49ers</a></td><td class="center " data-stat="boxscore_word"><a href="/boxscores/201610060sfo.htm">boxscore</a></td><td class="right " data-stat="pts_win"><strong>33</strong></td><td class="right " data-stat="pts_lose">21</td><td class="right " data-stat="yards_win">288</td><td class="right " data-stat="to_win">0</td><td class="right " data-stat="yards_lose">286</td><td class="right " data-stat="to_lose">3</td></tr>


In [7]:
class Week(Season):
    def __init__(self, year, week, games, upcoming = False):
        self.season = year       
        self.base_url = config['base_url']
        self.games = games
        url_week_dict = {
            "WildCard" : 18,
            "Divisional" : 19,
            "ConfChamp" : 20,
            "SuperBowl" : 21
        }
        url_week = url_week_dict[week] if week in url_week_dict.keys() else int(week)
        self.week_url = "{}/years/{}/week_{}.htm".format(self.base_url, self.season, url_week)    
        

In [8]:
week = Week(2016, '5', games)
week.season

2016

In [24]:
class Game():
    def __init__(self, year, week, game_summary_row, upcoming=False):
        
        # set these self variables first so all other fxns in init can run
        self.season = year
        self.week = week
        self.upcoming = upcoming
        self.base_url = config['base_url']
        self.game_url_append = game_summary_row.find("td", {"data-stat":"boxscore_word"}).a["href"]
        self.game_full_url = self.base_url + self.game_url_append        
        self.home_winner_col_name = "home_winner"
        self.game_link_col_name = "game_link"
        self.game_summary_row = game_summary_row
        self.game_summary_cols = game_summary_row.findAll('td')        
        self.game_summary_output_cols = [
            "year",
            "week_num",
            "game_date",
            self.game_link_col_name,
            self.home_winner_col_name, 
            "winner",
            "pts_win",
            "loser",
            "pts_lose"
        ]
        # end
        
        self.game_summary_dict = self.clean_game_summary(game_summary_row)  
        self.teams_list = self.get_teams(game_summary_row)
        self.game_html_page = self.get_game_page()
              
        self.favorite = self.get_favorite_or_underdog(favorite_flg = True)
        self.underdog = self.get_favorite_or_underdog(favorite_flg = False)
        self.home_team = self.get_home_or_vis(home_flg = True)
        self.vis_team = self.get_home_or_vis(home_flg = False)
        self.line = self.get_vegas_stats(stat = "line")
        self.over_under = self.get_vegas_stats(stat = "over_under")
        
    def get_home_or_vis(self, home_flg):
        
        game_summary_dict = self.game_summary_dict
        
        if self.upcoming == False:
            
            winner = game_summary_dict['winner']
            loser = game_summary_dict['loser']
            home_winner = game_summary_dict['home_winner']
            
            if home_flg == True:
                team =  winner if home_winner == True else loser
            else:
                team = loser if home_winner == True else winner 
                
        else:
            team = None
        
        return team
    
    def get_teams(self, game_summary_row):
        
        game_summary_dict = self.game_summary_dict
        teams_list = []
        
        for elem in ["winner", "loser"]:
            team = {
                "team" : game_summary_row.find('td', {'data-stat' : elem}).a.string, 
                "link" : game_summary_row.find('td', {'data-stat' : elem}).a['href']
            }
            
            if elem == "winner" and game_summary_dict['home_winner'] == True:
                team["home_flg"] = True 
            elif elem == "winner" and game_summary_dict['home_winner'] == False:
                team["home_flg"] = False
            elif elem == "loser" and game_summary_dict['home_winner'] == True:
                team["home_flg"] = False
            elif elem == "loser" and game_summary_dict['home_winner'] == False:
                team["home_flg"] = True
            teams_list.append(team)
        
        return teams_list
    
    def get_game_page(self):
        
        game_soup = get_html(self.game_full_url)
        return game_soup
    
    def get_vegas_stats(self, stat):
        
        game_html_page = self.game_html_page
        
        game_info_tbl = game_html_page.find("table", {"id":"game_info"})
        
        if stat == 'line':
            
            vegas_pattern = re.compile(r'Vegas Line')
            line_string = game_html_page.find('th', text=vegas_pattern).parent.td.string
            line_string_split = re.split('([\+-])', line_string)        
            return_val = float(line_string_split[-1])
            
        elif stat == "over_under":
            vegas_pattern = re.compile(r'Over/Under')
            ou_string = game_html_page.find('th', text=vegas_pattern).parent.td.contents[0]
            return_val = float(ou_string.strip())
            
        return return_val
    
    def get_favorite_or_underdog(self, favorite_flg):
        
        upcoming = self.upcoming
        game_page = self.game_html_page
        teams = [x["team"] for x in self.teams_list]
        game_info_tbl = game_page.find("table", {"id":"game_info"})
        vegas_pattern = re.compile(r'Vegas Line')
        line_string = game_info_tbl.find('th', text=vegas_pattern).parent.td.string
        line_string_split = re.split('([\+-])', line_string)
        sign = line_string_split[1]
        if favorite_flg == True and sign == "-":
            team = line_string_split[0].strip()
        elif favorite_flg == True and sign == "+":
            teams.remove(line_string_split[0].strip())
            team = teams[0]
        elif favorite_flg == False and sign == "-":
            teams.remove(line_string_split[0].strip())
            team = teams[0]
        elif favorite_flg == False and sign == "+":
            team = line_string_split[0].strip()
        
        #print(game_info_tbl)
#         if upcoming == False:
#             team = self.game_summary_dict['winner'] if 
        return team
    
    def clean_game_summary(self, game_summary_row):
        
        game_results_cols = self.game_summary_output_cols
        game_summary_cols = game_summary_row.findAll('td')
        
        game_summary_dict = {}
        
        for td in game_summary_cols:
            game_summary_dict = self.clean_game_summary_col(game_summary_dict, td)
        return game_summary_dict
    
    def clean_game_summary_col(self, game_summary_dict, td):
        
        data_stat = td.get('data-stat')
        if data_stat in ["game_date", "pts_lose"]:
            game_summary_dict[data_stat] = td.string
        if data_stat in ["winner", "loser"]:
            game_summary_dict[data_stat] = td.a.string
        if data_stat == "game_location":
            game_summary_dict[self.home_winner_col_name] = False if td.string == "@" else True
        if data_stat == "pts_win":
            game_summary_dict[data_stat] = td.strong.string

        return game_summary_dict

In [25]:
game = Game(2016, 5, week.games[0])

/Users/greg.bolla/.local/share/virtualenvs/nfl-stat-scraper-Jt7XuzPt/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [26]:
print(game.game_url_append)
print(game.teams_list)

/boxscores/201610060sfo.htm
[{'team': 'Arizona Cardinals', 'link': '/teams/crd/2016.htm', 'home_flg': False}, {'team': 'San Francisco 49ers', 'link': '/teams/sfo/2016.htm', 'home_flg': True}]


In [73]:
class Team(Game):
    def __init__(self, game_html, team_name, home_flg):
        
        self.team_table_id = "home" if home_flg == True else "vis"
        self.opp_table_id =  "vis" if home_flg == True else "home"
        self.home_flg = home_flg
        self.team = team_name
        self.game_html = game_html
        print(self.team_table_id)
        print(self.opp_table_id)
        
    
    def get_indiv_game_stats(self):
        game_stats_id = "team_stats"
        game_html = self.game_html
        game_html.find(id=final_tbl_id).tbody
        
    def clean_stats_table_row(self, row, stats_table_dict):
        stat_name_dict = {
            "First Downs" : "first_downs",
            "Rush" : "rush_att",
            "Yds" : "rush_yds",
            "TDs" : "rush_tds",
            "Cmp" : "pass_comp",
            "Att" : "pass_att",
            "Yd" : "pass_yds",
            "TD" : "pass_tds",
            "INT" : "pass_int",
            "Sacked" : "sacks",
            "Net Pass Yards" : "net_pass_yds",
            "Total Yards" : "total_yds",
            "Fumbles" : "fumbles",
            "Lost" : "fumbles_lost",
            "Turnovers" : "turnovers",
            "Penalties" : "penalties",
            "Yards" : "penalty_yds",
            "Third Down Conv." : "third_down_conv",
            "Fourth Down Conv."  : "fourth_down_conv",
            "Time of Possession" : "time_of_poss"
        }    
        team_id = "{}_stat".format(self.team_table_id)
        opp_id = "{}_stat".format(self.opp_table_id)
        stat = tr.th.string
        team_stat = tr.find('td', {"data-stat" : team_id}).string
        opp_stat = tr.find('td', {"data-stat" : opp_id}).string
        
        if "-" in stat:
            stat_name_split = stat.split("-")
            team_stat_split = team_stat.split("-")
            opp_stat_split = opp_stat.split("-")

            for idx, stat in enumerate(stat_name_split):
                stdized_stat = stat_name_dict[stat]
                team_stat = team_stat_split[idx]
                opp_stat = opp_stat_split[idx]
                stats_table_dict["team_{}".format(stdized_stat)] = team_stat
                stats_table_dict["opp_{}".format(stdized_stat)] = opp_stat
        else:
            stdized_stat = stat_name_dict[stat]
        

        stats_table_dict["team_{}".format(stdized_stat)] = team_stat
        stats_table_dict["opp_{}".format(stdized_stat)] = opp_stat

        return stats_table_dict

In [74]:
arz = game.teams_list[0]

team = Team(game.game_html_page, arz["team"], arz["home_flg"])

vis
home


In [76]:
game_stats_id = "team_stats"
game_html = team.game_html


stats_table = game_html.find(id=game_stats_id).tbody
stats_table_rows = stats_table.findAll('tr')

stats_table_dict = {}
for tr in stats_table_rows:
    #print(tr)
    stats_table_dict = team.clean_stats_table_row(tr, stats_table_dict)
    
print(stats_table_dict)




{'team_first_downs': '17', 'opp_first_downs': '25', 'team_rush_att': '37', 'opp_rush_att': '36', 'team_rush_yds': '172', 'opp_rush_yds': '151', 'team_rush_tds': '2', 'opp_rush_tds': '2', 'team_pass_comp': '11', 'opp_pass_comp': '18', 'team_pass_att': '28', 'opp_pass_att': '31', 'team_pass_yds': '124', 'opp_pass_yds': '162', 'team_pass_tds': '2', 'opp_pass_tds': '1', 'team_pass_int': '0', 'opp_pass_int': '2', 'team_sacks': '1', 'opp_sacks': '7', 'team_penalty_yds': '50', 'opp_penalty_yds': '34', 'team_net_pass_yds': '116', 'opp_net_pass_yds': '135', 'team_total_yds': '288', 'opp_total_yds': '286', 'team_fumbles': '2', 'opp_fumbles': '2', 'team_fumbles_lost': '0', 'opp_fumbles_lost': '1', 'team_turnovers': '0', 'opp_turnovers': '3', 'team_penalties': '7', 'opp_penalties': '4', 'team_third_down_conv': '5-16', 'opp_third_down_conv': '4-14', 'team_fourth_down_conv': '0-0', 'opp_fourth_down_conv': '1-1', 'team_time_of_poss': '31:44', 'opp_time_of_poss': '28:16'}


In [72]:
stats_table_dict

{'team_first_downs': '17',
 'opp_first_downs': '25',
 'team_rush_att': '37',
 'opp_rush_att': '37',
 'team_rush_yds': '172',
 'opp_rush_yds': '172',
 'team_rush_tds': '2',
 'opp_rush_tds': '2',
 'team_pass_comp': '11',
 'opp_pass_comp': '11',
 'team_pass_att': '28',
 'opp_pass_att': '28',
 'team_pass_yds': '124',
 'opp_pass_yds': '124',
 'team_pass_tds': '2',
 'opp_pass_tds': '2',
 'team_pass_int': '0',
 'opp_pass_int': '0',
 'team_sacks': '1',
 'opp_sacks': '1',
 'team_penalty_yds': '50',
 'opp_penalty_yds': '50',
 'team_net_pass_yds': '116',
 'opp_net_pass_yds': '135',
 'team_total_yds': '288',
 'opp_total_yds': '286',
 'team_fumbles': '2',
 'opp_fumbles': '2',
 'team_fumbles_lost': '0',
 'opp_fumbles_lost': '0',
 'team_turnovers': '0',
 'opp_turnovers': '3',
 'team_penalties': '7',
 'opp_penalties': '7',
 'team_third_down_conv': '5-16',
 'opp_third_down_conv': '4-14',
 'team_fourth_down_conv': '0-0',
 'opp_fourth_down_conv': '1-1',
 'team_time_of_poss': '31:44',
 'opp_time_of_poss':

In [ ]:
teams.remove(line_string_split[0].strip())

In [ ]:
line_string_split[-1]